In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.porter import PorterStemmer

In [2]:
def indiscernibility(matrix,y):
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    y=list(df.columns)
    grouped_df=df.groupby(y)
    
    ind_R=list(list())
    for key, item in grouped_df:
#         print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

def encoding_discourse_type(x):
    if x=="Lead":
        return 0
    if x=="Position":
        return 1
    if x=="Evidence":
        return 2
    if x=="Claim":
        return 3
    if x=="Concluding Statement":
        return 4
    if x=="Counterclaim":
        return 5
    if x=='Rebuttal':
        return 6
    
def stemming_stopwords_removing(df):
    corpus=[]
    for i in range(len(df)):
        review=re.sub('[^a-zA-Z]',' ',df["discourse_text"][i])
        review=review.lower()
        review=review.split()
        ps=PorterStemmer()
        all_stopwords=stopwords.words("english")
        review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
        review=' '.join(review)
        corpus.append(review)
    return corpus

# storing the total occurrence.......
def get_total_index_words(corpus):
    index_word={}
    for i in corpus:
        s=i.split()
        for j in s:
            if j not in index_word:
                index_word[j]=1
            else:
                index_word[j]+=1
    return index_word
    
def get_values(dataset,threshold=1):
    
    # taking sample of 20 documents for lead category....
    df = dataset

    total_corpus = stemming_stopwords_removing(df)
    # print(total_corpus)

    # getting total index words and their count in the taken sample as a dict
    total_index_words = get_total_index_words(total_corpus)
    # print(len(lead_index_words))

    # Creating a list of total keywords before filtering..
    total_keywords = list(total_index_words.keys())

    # Creating a matrix of width equals len(lead_keywords)
    matrix=np.zeros((len(df),len(total_keywords)))

    
    # Storing occurrence of each term in each document respectively
    for i in range(len(total_corpus)):
        s = total_corpus[i].split()
        for h in s:
            j = total_keywords.index(h)
            matrix[i,j] += 1


    # Storing their weights....
    weighted_matrix = np.copy(matrix)
    
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            weighted_matrix[i,j] = weighted_matrix[i,j] / total_index_words[total_keywords[j]]
    #            print(weighted_matrix[i,j])



    # FILTERING WEIGHTS with a threshold.......
    valid_index = []
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            if weighted_matrix[i,j] >= threshold:
                valid_index.append(j)

    # removing duplicates and storing them in a list.......    
    valid_index = list(set(valid_index))


    # # Storing the final keywords.... 
    valid_index_words = []
    for i in range(len(valid_index)):
        valid_index_words.append(total_keywords[valid_index[i]])
    # print(valid_lead_index_words)

    return total_keywords,total_index_words,matrix, weighted_matrix, valid_index, valid_index_words

def get_test_matrix(dataset,total_sample_keywords,threshold=1):
      # taking sample of 20 documents for lead category....
    df = dataset

    total_corpus = stemming_stopwords_removing(df)
    # print(total_corpus)

    # getting total index words and their count in the taken sample as a dict
    total_index_words = get_total_index_words(total_corpus)


    # Creating a list of total keywords before filtering..
    total_keywords = total_sample_keywords
    # print(lead_keywords)

    # Creating a matrix of width equals len(lead_keywords)
    matrix=np.zeros((len(df),len(total_keywords)))
    print(len(df),len(total_keywords))

    
    # Storing occurrence of each term in each document respectively
    for i in range(len(total_corpus)):
        s = total_corpus[i].split()
        for h in s:
            try:
                j = total_keywords.index(h)
                matrix[i,j] += 1
            except:
                continue
            
    # Storing their weights....
    weighted_matrix = np.copy(matrix)
    
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            try:
                weighted_matrix[i,j] = weighted_matrix[i,j] / total_index_words[total_keywords[j]]
            except:
                continue


    # FILTERING WEIGHTS with a threshold.......
    valid_index = []
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            if weighted_matrix[i,j] >= threshold:
                valid_index.append(j)

    # removing duplicates and storing them in a list.......    
    valid_index = list(set(valid_index))
    


    # # Storing the final keywords.... 
    valid_index_words = []
    for i in range(len(valid_index)):
        valid_index_words.append(total_keywords[valid_index[i]])
    # print(valid_lead_index_words)

    return total_keywords,total_index_words,matrix, weighted_matrix, valid_index, valid_index_words


In [3]:
def single_element_indiscernibility(matrix,index):
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    y=list(df.columns)
    
    grouped_df=df.groupby(y[index])
    
    ind_R=list(list())
    for key, item in grouped_df:
    #print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

def indiscernibility(matrix,y):
    if len(y)==0:
        return []
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    grouped_df=df.groupby(y)
    ind_R=list(list())
    for key, item in grouped_df:
    #print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

    

In [4]:
train=pd.read_csv("train.csv")
train_head=train.head(500)
train_tail=train.tail(500)
train=pd.concat([train_head,train_tail])
train.reset_index(inplace=True)
train

,index,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...,...
995,144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
996,144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
997,144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838
998,144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...


In [5]:
# Now i will send my training data to get its list of final keywords after eliminating useles words from our list
# using weighted matrix .....

train_total_keywords,train_total_index_words, train_matrix, train_weighted_matrix, train_valid_index, train_final_keywords=get_values(train)

In [6]:
print(len(train_weighted_matrix))
c_attr=train_weighted_matrix.copy()

1000


In [7]:
train["Y"]=train["discourse_type"].copy()
train["Y"]=train["Y"].apply(lambda x: encoding_discourse_type(x))
y=train["Y"].values
print(len(y))
dec_attr=y.copy()

1000


In [8]:
# NOW REMOVING IDENTICAL COLUMNS FROM OUR MATRIX INPLACE.......
print((c_attr))
print("shape of old training attribute",c_attr.shape)
grp=np.column_stack(c_attr)
temp=pd.DataFrame(grp)
# print(temp.shape)

temp_grp=indiscernibility(temp,list(temp.columns))
# print(temp_grp)

unique_columns=[]
for i in range(len(temp_grp)):
    unique_columns.append(temp_grp[i][0])
    
unique_columns.sort()
print()
# print("Unique columns:",unique_columns)
print("\nNo of unique columns:",len(unique_columns))

unique_col_table=list(list())
for i in unique_columns:
    unique_col_table.append(list(grp[i]))
unique_col_table=np.array(unique_col_table)
# print("\nunique_col_table:")
print(unique_col_table)
print("\n unique_col_table shape",unique_col_table.shape)

new_train_attributes=np.column_stack(unique_col_table)

print("\nnew train attributes after preserving only one repetitve element:\n",new_train_attributes)
print("\nshape of new train attributes:",new_train_attributes.shape)
c_attr=new_train_attributes.copy()

repetitive_columns=list(set(list(range(len(c_attr[0]))))-set(unique_columns))
print("number of repeated columns are: ", len(repetitive_columns))

[[0.5        0.1        0.03703704 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         1.        ]]
shape of old training attribute (1000, 2076)


No of unique columns: 1564
[[0.5        0.         0.         ... 0.         0.         0.        ]
 [0.1        0.         0.         ... 0.         0.         0.        ]
 [0.03703704 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.5        0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]

 unique_col_table shape (1564, 1000)


In [9]:
print(c_attr.shape)
print(y.shape)

(1000, 1564)
(1000,)


In [10]:
from sklearn import datasets
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from frlearn.base import probabilities_from_scores, select_class
from frlearn.classifiers import FRNN
from frlearn.feature_preprocessors import RangeNormaliser

In [11]:
# Split into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(c_attr, y, stratify=y, random_state=0)





In [12]:
y_train

array([2, 2, 2, 3, 4, 3, 2, 4, 3, 3, 3, 1, 1, 2, 0, 3, 3, 3, 6, 2, 2, 2,
       3, 2, 0, 2, 3, 5, 3, 6, 4, 2, 2, 1, 2, 2, 2, 3, 3, 1, 2, 4, 2, 3,
       0, 3, 3, 1, 3, 3, 4, 1, 3, 3, 1, 1, 3, 2, 3, 0, 2, 3, 2, 2, 0, 2,
       3, 3, 2, 2, 4, 2, 1, 3, 2, 1, 0, 3, 2, 1, 0, 2, 3, 2, 3, 0, 2, 3,
       0, 2, 1, 3, 3, 3, 1, 0, 4, 3, 2, 3, 2, 2, 3, 2, 1, 4, 4, 3, 0, 3,
       2, 3, 3, 2, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 4, 3, 2, 1, 3, 4, 3, 3,
       2, 4, 2, 4, 2, 2, 0, 2, 3, 3, 2, 4, 3, 5, 2, 2, 3, 2, 0, 0, 3, 2,
       4, 4, 4, 2, 2, 3, 2, 2, 4, 2, 2, 1, 3, 2, 3, 3, 2, 2, 2, 0, 1, 3,
       0, 1, 3, 2, 2, 1, 0, 3, 3, 3, 1, 4, 2, 3, 1, 3, 2, 2, 2, 4, 3, 3,
       1, 1, 5, 1, 2, 3, 1, 3, 3, 2, 3, 2, 2, 0, 1, 3, 0, 4, 2, 2, 4, 3,
       2, 3, 0, 2, 2, 2, 3, 3, 3, 3, 4, 3, 0, 2, 4, 3, 2, 3, 3, 1, 1, 2,
       1, 2, 3, 2, 3, 2, 4, 3, 3, 2, 4, 3, 2, 2, 2, 2, 2, 2, 3, 2, 0, 2,
       3, 2, 3, 6, 3, 3, 3, 3, 2, 2, 0, 4, 3, 3, 5, 3, 1, 2, 4, 3, 3, 5,
       2, 2, 1, 3, 3, 2, 0, 2, 3, 3, 2, 3, 1, 2, 3,

In [13]:
# Create an instance of the FRNN classifier, construct the model, and query on the test set.
clf = FRNN(preprocessors=(RangeNormaliser(), ))
model = clf(X_train, y_train)
scores = model(X_test)

In [14]:
# Convert scores to probabilities and calculate the AUROC.
probabilities = probabilities_from_scores(scores)
auroc = roc_auc_score(y_test, probabilities, multi_class='ovo')
print('AUROC:', auroc)

# Select classes with the highest scores and calculate the accuracy.
probabilities = probabilities_from_scores(scores)
auroc = roc_auc_score(y_test, probabilities, multi_class='ovo')
print(len(probabilities[0]))
print(probabilities)
print('AUROC:', auroc)

AUROC: 0.7570373869371759
7
[[0.14287906 0.1430104  0.142788   ... 0.14278691 0.14278143 0.14268695]
 [0.14290388 0.14302379 0.14278631 ... 0.14277629 0.14278307 0.14266518]
 [0.14288094 0.143019   0.14277448 ... 0.1427848  0.14279966 0.14266866]
 ...
 [0.14290179 0.14299082 0.14283401 ... 0.14277546 0.14278601 0.14264363]
 [0.14287319 0.14307461 0.14276218 ... 0.14277611 0.14279623 0.14267299]
 [0.14290386 0.14301149 0.1428255  ... 0.14278934 0.14275216 0.1426576 ]]
AUROC: 0.7570373869371759
